# IESB - Graduacao - CIA035 - Malaria CNN

In [1]:
import os

import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, DataLoader
from torch import nn, optim

In [2]:
# Verificando o diretório com as imagens
img_dir = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"
print(os.listdir(img_dir))

['Parasitized', 'Uninfected']


In [3]:
# Temos GPU?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [4]:
# Transformações que serão aplicadas às imagens
image_transform = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], 
                                                           [0.5, 0.5, 0.5])])

In [5]:
# Criando nosso conjunto de dados com base nas imagens
data = datasets.ImageFolder(img_dir,transform=image_transform)

In [6]:
# Quais as classes possíveis?
data.classes

['Parasitized', 'Uninfected']

In [7]:
# Verificando o tamanho do conjunto de dados
len(data)

27558

In [8]:
# Dividindo os dados em treino e teste
train_data, test_data = random_split(data, (20000, 7558))

In [9]:
# Definindo os loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [10]:
# Definir LeNet5

# Cálculo do tamanho da saída de cada convolução
# outputOfEachConvLayer = [(size + 2*padding - kernel_size) / stride] + 1

model = nn.Sequential(
    # Primeira convolução
    nn.Conv2d(in_channels=3, out_channels=18, kernel_size=5, padding=0, stride=1),
    # Função de ativação
    nn.Tanh(),
    # Average Pooling
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # Segunda convolução
    nn.Conv2d(in_channels=18, out_channels=48, kernel_size=5, padding=0, stride=1),
    # Função de ativação
    nn.Tanh(),
    # Average Pooling
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    # Convertendo a imagem em estrutura plana
    nn.Flatten(),
    
    # Iniciando a rede fully connected
    # in_features = 1200
    nn.Linear(in_features=48*5*5, out_features=120),
    nn.Tanh(),
    nn.Linear(in_features=120, out_features=84),
    nn.Tanh(),
    nn.Linear(in_features=84, out_features=2)
)

model.to(device)

Sequential(
  (0): Conv2d(3, 18, kernel_size=(5, 5), stride=(1, 1))
  (1): Tanh()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(18, 48, kernel_size=(5, 5), stride=(1, 1))
  (4): Tanh()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten()
  (7): Linear(in_features=1200, out_features=120, bias=True)
  (8): Tanh()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Tanh()
  (11): Linear(in_features=84, out_features=2, bias=True)
)

In [11]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

error_function.to(device)

CrossEntropyLoss()

In [12]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 10

# Colocando o modelo em modo de treinamento
model.train()


# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Monitorando o training loss
    train_loss = 0.0
    
    # Obtendo dados e respostas
    for data, target in train_loader:
        
        # Enviando os dados para GPU, se existir
        data, target = data.to(device), target.to(device)
    
        # Foward Propagation (passando os dados de treino pela rede)
        outputs = model(data)
        # Calculando o erro
        loss = error_function(outputs, target)
       
        # Back Propagation
        # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
        optimizer.zero_grad()
        # Calcular os novos pesos
        loss.backward()
        # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
        optimizer.step()
        
        # Atualizando o training loss
        train_loss += loss.item() * data.size(0)
        
    # Calculando a média de erro por epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(i+1, train_loss))

Epoch: 1 	Training Loss: 0.602981
Epoch: 2 	Training Loss: 0.567013
Epoch: 3 	Training Loss: 0.513825
Epoch: 4 	Training Loss: 0.424428
Epoch: 5 	Training Loss: 0.345050
Epoch: 6 	Training Loss: 0.290407
Epoch: 7 	Training Loss: 0.259349
Epoch: 8 	Training Loss: 0.233517
Epoch: 9 	Training Loss: 0.213213
Epoch: 10 	Training Loss: 0.202067


In [13]:
# Variaveis para controlar os acertos das previsões da rede
# e  calcular a acurácia
correct = 0
total = 0

# Vamos colocar o modelo em modo de avaliação/teste
model.eval()

# Obtendo dados e respostas
for data, target in test_loader:
    
    # Enviando os dados para GPU, se existir
    data, target = data.to(device), target.to(device)    
    
    output = model(data)
    
    for index, i in enumerate(output):
        if torch.argmax(i) == target[index]:
            correct += 1
        total += 1

In [14]:
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.897
